# 0-install dependencies

In [2]:
!pip install tensorflow==1.15.0 tensorflow-gpu==1.15.0 stable_baselines gym box2d-py --user

  Using cached tensorflow-1.15.0-cp37-cp37m-win_amd64.whl (295.1 MB)
  Using cached tensorflow_gpu-1.15.0-cp37-cp37m-win_amd64.whl (294.5 MB)
  Using cached stable_baselines-2.10.2-py3-none-any.whl (240 kB)
  Using cached box2d-py-2.3.8.tar.gz (374 kB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Using cached gast-0.2.2-py3-none-any.whl
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp37-cp37m-win_amd64.whl size=490235 sha256=bb3461d1dc2ccc1383730057a2aab3eebb0f5f1965c63c9a4b41cb759595ba52
  Stored in directory: c:\users\leilaa\appdata\local\pip\cache\wheels\3b\b0\14\ed70a19a3f74b1d57d20eaea638c15e14878188d0a71be22ee
Successfully built box2d-py


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\users\leilaa\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:
environment_name = 'LunarLander-v2'

# 1-Test random environment

In [7]:
env = gym.make(environment_name)

In [9]:
episodes = 10
for episode in range(1 , episodes+1):
    state = env.reset()
    score = 0
    done = False
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{}, Score:{}'.format(episode,score))
env.close()
    

Episode:1, Score:-89.88481528885208
Episode:2, Score:-156.24587398078611
Episode:3, Score:-121.27232915293243
Episode:4, Score:-78.2633983354658
Episode:5, Score:-122.7546259173493
Episode:6, Score:-58.73159275774646
Episode:7, Score:-358.2412949065117
Episode:8, Score:-294.5442546781793
Episode:9, Score:-106.280114643087
Episode:10, Score:-317.95126531107474


# 2- Build and Train the model

In [11]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = A2C('MlpPolicy', env, verbose=1)
#MlpPolicy , CnnPolicy could use for image based environment, LstmPolicy for Recurrent neural network for time series environment

C:\Users\leilaa\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 8000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device


In [13]:
model.learn(total_timesteps=10000)

------------------------------------
| time/                 |          |
|    fps                | 741      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.31    |
|    explained_variance | -0.0444  |
|    learning_rate      | 0.0007   |
|    n_updates          | 299      |
|    policy_loss        | 6.46     |
|    value_loss         | 45.1     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 677      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.24    |
|    explained_variance | 0.00257  |
|    learning_rate      | 0.0007   |
|    n_updates          | 399      |
|    policy_loss        | -5.35    |
|    value_loss         | 22.9     |
-

-------------------------------------
| time/                 |           |
|    fps                | 620       |
|    iterations         | 1700      |
|    time_elapsed       | 13        |
|    total_timesteps    | 8500      |
| train/                |           |
|    entropy_loss       | -0.57     |
|    explained_variance | -8.07e-05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 1899      |
|    policy_loss        | 1.52      |
|    value_loss         | 11.3      |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 623       |
|    iterations         | 1800      |
|    time_elapsed       | 14        |
|    total_timesteps    | 9000      |
| train/                |           |
|    entropy_loss       | -0.7      |
|    explained_variance | -5.47e-05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 1999      |
|    policy_loss        | -0.327    |
|    value_l

# 3-Save and test the model

In [16]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

C:\Users\leilaa\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


In [17]:
model.save('A2C_model')

In [19]:
del model

In [25]:
model = A2C.load('A2C_model', env=env)

In [28]:
obs = env.reset()
while True:
    action, _state = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()

KeyboardInterrupt: 